In [1]:
# For now aerobulk installation only worked on greene not laptop
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from aerobulk.flux import noskin_np, skin_np, noskin, skin

%load_ext autoreload
%autoreload 3

In [2]:
# atm has 576 * 360 grid
# ocn has 3600 * 2700 grid

# Read in data
from mlflux.utils import sort_longitude

atm = xr.open_dataset('../../data/Global/GFDL_CM2p6_control_atm_0101.nc', chunks={})
atm['t_ref'] = atm['t_ref'] 

ocn = xr.open_dataset('../../data/Global/GFDL_CM2p6_control_ocn_0101.nc', chunks={})
ocn = ocn.isel(time=0)

ocn = ocn.rename({'xt_ocean': 'lon', 'yt_ocean': 'lat'})
ocn = sort_longitude(ocn)
atm['sst'] = ocn.surface_temp.interp(lon=atm['grid_xt'],lat=atm['grid_yt']) + 273.15
ocn = ocn.rename({'lon': 'xt_ocean', 'lat': 'yt_ocean'})

ocn = ocn.rename({'xu_ocean': 'lon', 'yu_ocean': 'lat'})
ocn = sort_longitude(ocn)
atm['usurf'] = ocn.usurf.interp(lon=atm['grid_xt'],lat=atm['grid_yt'])
atm['vsurf'] = ocn.vsurf.interp(lon=atm['grid_xt'],lat=atm['grid_yt'])
ocn = ocn.rename({'lon': 'xu_ocean', 'lat': 'yu_ocean'})

In [3]:
# To make sure that the surface pressure field is in a valid range
atm["ps_adjust"] = atm["ps"].where((atm["ps"] < 110000) & (atm["ps"] > 80000), 101000)

# To make sure that nans match the sst land mask
sst_wet_mask = ~np.isnan(atm['sst'])
ds = atm.where(sst_wet_mask.compute())

In [6]:
# Change the grid_yt range from 40 to 39 generates all nans
ds = atm.where(sst_wet_mask.compute()).isel(grid_xt=slice(None,None), grid_yt=slice(39,-1))

ql, qh, taux, tauy, evap = \
noskin(sst=ds.sst, t_zt=ds.t_ref, 
       hum_zt=ds.q_ref, u_zu=ds.wind, 
       v_zu=ds.wind*0, slp=ds.ps_adjust, 
       algo='coare3p0', zt=ds.wind/ds.wind*2., zu=ds.wind/ds.wind*10.,
       input_range_check=True)

print(qh.values)

/ext3/miniconda3/lib/python3.11/site-packages/aerobulk/flux.py:280: UserWarning: Checking for misaligned nans and values outside of the valid range is performed by default, but reduces performance. 
If you are sure your data is valid you can deactivate these checks by setting `input_range_check=False`
  warnings.warn(performance_msg)


[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
